No costs!  Bases and jobs have simple sum requirement.  Mods use decision trees to define eligibility.

In [ ]:


function apply_ability(a::Spud, b::Spud)::Array{Spud}
    if a.a1 == ability_none
        return [a, b]
    else
        return ability_fx[a.a1](a, b)
    end
end


function eval_battle(a::Spud, b::Spud)::Int64
    utb = compare_int_list(spud_utb_seq(a), spud_utb_seq(b), 0) # universal tiebreaker
    if utb == 0
        return 0
    end
    a_ability_first = compare_int_list([a.a1, a.s], [b.a1, b.s], utb)
    if a_ability_first ==1
        if a.a1 != ability_none
            res = apply_ability(a, b)
            a = res[1]
            b = res[2]
        end
        if b.a1 != ability_none
            res = apply_ability(b, a)
            b = res[1]
            a = res[2]
        end
    end
    if a_ability_first == -1
        if b.a1 != ability_none
            res = apply_ability(b, a)
            b = res[1]
            a = res[2]
        end
        if a.a1 != ability_none
            res = apply_ability(a, b)
            a = res[1]
            b = res[2]
        end
    end
    #println(a)
    #println(b)
    a_finds = eval_finds(a, b, utb)==1
    b_finds = eval_finds(b, a, utb)==1
    melee_win = eval_melee(a, b, 0, 0)
    luck_win = compare_int_list([a.l, melee_win], [b.l, -melee_win], 0)
    if melee_win ==1 && luck_win ==1
        return 1
    end
    if melee_win == -1 && luck_win == -1
        return -1
    end
    if a_finds && b_finds
        return melee_win
    end
    if a_finds && !b_finds
        return 1
    end
    if !a_finds && b_finds
        return -1
    end
    if !a_finds && !b_finds
        return luck_win
    end
end

In [ ]:
const discount = 20

function rename_base(a::SpudBase, s::String)::SpudBase
    return SpudBase(s, a.h,a.f,a.l,a.p,a.r,a.s,a.a1,a.cost)
end

function rename_job(a::SpudMod, s::String)::SpudMod
    return SpudMod(s, a.h,a.f,a.l,a.p,a.r,a.s,a.a1,a.minh,a.minf,a.minl,a.minp,a.minr,a.mins,a.cost)
end

function check_validity(a::Spud)::Bool
    return minimum([a.h,a.f,a.l,a.p,a.r,a.s]) >= MNV
end

function combine_base_job(b::SpudBase, j::SpudMod)::Spud
    v = [b.h,b.f,b.l,b.p,b.r,b.s]
    c = b.cost
    a1 = b.a1
    if a1 == 999
        a1 = j.a1
    end
    w = [j.h,j.f,j.l,j.p,j.r,j.s]
    m = [j.minh,j.minf,j.minl,j.minp,j.minr,j.mins]
    c2 = j.cost
    if sum(v .< m) == 0
        c2 -= discount
    end
    z = v .+ w
    z[z .> MXV] .= MXV
    return Spud(string(j.name, " ", b.name),z[1],z[2],z[3],z[4],z[5],z[6],a1,c+c2)
end


function combine_spud_mod(b::Spud, j::SpudMod)::Spud
    v = [b.h,b.f,b.l,b.p,b.r,b.s]
    c = b.cost
    a1 = b.a1
    if a1 == 999
        a1 = j.a1
    end
    w = [j.h,j.f,j.l,j.p,j.r,j.s]
    m = [j.minh,j.minf,j.minl,j.minp,j.minr,j.mins]
    c2 = j.cost
    # require spud to meet minimums
    if sum(v .< m) > 0 || minimum(v .+ w) < MNV
        return b
    end
    z = v .+ w
    z[z .> MXV] .= MXV
    return Spud(string(j.name, " ", b.name),z[1],z[2],z[3],z[4],z[5],z[6],a1,c+c2)
end

In [ ]:
function get_spuds_with_value(scores::Array{Int64}, vmin::Int64, vmax::Int64)::Array{Spud}
    inds = findall(scores .>= vmin .&& scores .<= vmax)
    return map(x -> Spud("spud",x[1],x[2],x[3],x[4],x[5],x[6],ability_none,scores[x]), inds)
end

function choice_wo(x::Array, n::Int64)::Array
    w = [rand() for i in 1:length(x)]
    return x[sortperm(w)][1:n]
end

#### Load pre-computed scores (from game_designL)

In [520]:
scores = matread("temp_desE_scores400.mat")["scores"];
global_ne = get_spuds_with_value(scores, 95, 100);

In [524]:
#wt = [1,1,1,1,1,1]
wt = [13,14,19,10,10,10]
sums = [sum(spud_to_vec(s) .* wt) for s in global_ne];
minimum(sums), maximum(sums), minimum(sums)/maximum(sums)

(286, 479, 0.5970772442588727)

In [525]:
spuds_to_df(global_ne[sortperm(sums)[1:8]])

Row,name,h,f,l,p,r,s,a1,cost
,String,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64
1,spud,1,6,1,8,1,8,999,95
2,spud,1,6,1,1,8,8,999,97
3,spud,1,6,1,1,7,9,999,96
4,spud,7,1,7,2,1,2,999,95
5,spud,7,1,7,1,2,2,999,96
6,spud,7,1,7,1,1,3,999,96
7,spud,1,7,1,1,8,7,999,96
8,spud,1,7,1,1,7,8,999,97


In [526]:
spuds_to_df(global_ne[sortperm(-sums)[1:5]])

Row,name,h,f,l,p,r,s,a1,cost
,String,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64
1,spud,4,9,9,2,9,2,999,100
2,spud,9,1,4,9,9,9,999,97
3,spud,3,9,9,2,9,3,999,100
4,spud,4,9,9,8,3,1,999,99
5,spud,4,9,9,3,8,1,999,98


In [545]:
# get all vectors with sum 293
temp = findall(scores .>= -999);
sums = [t[1]*wt[1]+t[2]*wt[2]+t[3]*wt[3]+t[4]*wt[4]+t[5]*wt[5]+t[6]*wt[6] for t in temp];
temp = temp[sums .<= 274 .&& sums .>= 265];
bases = map((x, i) -> Spud(string("b", i),x[1],x[2],x[3],x[4],x[5],x[6],ability_none,0), temp, 1:length(temp));
length(bases)

11113

In [546]:
base_vs = copy(transpose(reduce(hcat, map(spud_to_vec, bases))));

In [547]:
maximum([scores[CartesianIndex((base_vs[i, :]...))] for i in 1:size(base_vs)[1]])

93

In [548]:
bases[sortperm(-[scores[CartesianIndex((base_vs[i, :]...))] for i in 1:size(base_vs)[1]])][1:10]

10-element Vector{Spud}:
 Spud("b26", 7, 1, 7, 1, 1, 1, 999, 0)
 Spud("b2060", 8, 1, 6, 1, 1, 2, 999, 0)
 Spud("b33", 6, 1, 8, 1, 1, 1, 999, 0)
 Spud("b63", 8, 1, 6, 2, 1, 1, 999, 0)
 Spud("b337", 8, 1, 6, 1, 2, 1, 999, 0)
 Spud("b10553", 1, 5, 1, 1, 8, 8, 999, 0)
 Spud("b10068", 1, 5, 1, 8, 1, 8, 999, 0)
 Spud("b11065", 1, 5, 1, 1, 7, 9, 999, 0)
 Spud("b2064", 6, 1, 7, 1, 1, 2, 999, 0)
 Spud("b341", 6, 1, 7, 1, 2, 1, 999, 0)

In [555]:
# consider a random target
target = rand(global_ne)

Spud("spud", 7, 1, 5, 7, 3, 6, 999, 95)

In [556]:
# find a close base
tarv = spud_to_vec(target)
dists = [sum(map(abs, base_vs[i, :] .- tarv)) for i in 1:length(bases)];
dom = [sum(base_vs[i, :] .- tarv .> 0)==0 for i in 1:length(bases)];
cand_inds = findall(dists .== minimum(dists) .&& dom)
cand_ind = rand(cand_inds)

8602

In [557]:
length(cand_inds)

4

In [558]:
b = bases[cand_ind]
bv = base_vs[cand_ind, :]
b

Spud("b8602", 6, 1, 1, 7, 3, 6, 999, 0)

In [559]:
diff = tarv .- bv

6-element Vector{Int64}:
 1
 0
 4
 0
 0
 0

In [560]:
final_vs = base_vs .+ transpose(diff)
final_vs[final_vs .> MXV] .= MXV;
cs = map(x -> scores[CartesianIndex(x...)], eachrow(final_vs))
mean(cs .< MXS)

0.7026005579051561

In [ ]:
## come up with min and max criteria that select a region of base_vs which are eligible (cs <= MXS)
#mn = [1,1,1,1,1,1]
#mx = [9,9,9,9,9,9]
mn = [1,1,1,1,1,1]
mx = [9,9,9,9,9,9]
filt = map(x -> sum(x .< mn) == 0 && sum(x .> mx) == 0, eachrow(base_vs))
maximum(cs[filt])

In [ ]:
filtp = filt[sortperm(-cs)]
hcat(base_vs,cs)[sortperm(-cs),:][filtp,:][1:10,:]

In [ ]:
mn = [3,1,1,1,1,1]
mx = [9,9,9,9,9,9]
filt = map(x -> sum(x .< mn) == 0 && sum(x .> mx) == 0, eachrow(base_vs))
maximum(cs[filt]),sum(filt .&& cs .<= MXS)/sum(cs .<= MXS)

In [ ]:
mn = [3,1,4,1,1,1]
mx = [9,9,9,9,9,9]
filt = map(x -> sum(x .< mn) == 0 && sum(x .> mx) == 0, eachrow(base_vs))
maximum(cs[filt]),sum(filt .&& cs .<= MXS)/sum(cs .<= MXS)

In [ ]:
filtp = filt[sortperm(-cs)]
hcat(base_vs,cs)[sortperm(-cs),:][filtp,:][1:10,:]

In [ ]:
mn = [3,1,4,1,1,1]
mx = [9,9,9,3,9,9]
filt = map(x -> sum(x .< mn) == 0 && sum(x .> mx) == 0, eachrow(base_vs))
maximum(cs[filt]),sum(filt .&& cs .<= MXS)/sum(cs .<= MXS)

In [ ]:
filtp = filt[sortperm(-cs)]
hcat(base_vs,cs)[sortperm(-cs),:][filtp,:][1:10:200,:]